In [155]:
%matplotlib inline
import numpy as np
import pandas as pd
import math

#Imports sklearn ------------------------------------------

#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

#Imports nltk ---------------------------------------------
import nltk
from nltk.stem.snowball import SnowballStemmer

from bs4 import BeautifulSoup



# Carregando Dataset de Perguntas e Respostas

In [156]:
def load( path ):
    df = pd.read_csv(path);
    
    perguntas = [];
    respostas = [];
    id_repostas = [];

    for row in df.iterrows():
        row = row[1];
        perguntas.append(BeautifulSoup(row.get("perguntas"), 'html.parser').get_text());
        respostas.append(BeautifulSoup(row.get("respostas"), 'html.parser').get_text());
        id_repostas.append(row.get("idResp"));
        
    return perguntas, respostas, id_repostas; 

perguntas, respostas, id_repostas = load("./QueryResults.csv");

In [157]:
perguntas[0], respostas[0]

('Diversas linguagens possuem um meio de se obter a "identidade" de um objeto, i.e. um número [inteiro] que é único para cada objeto, tal que objetos diferentes tenham identidades diferentes. Exemplos:\n\nJava (System.identityHashCode)\n\n\nTanto quanto razoavelmente prático, o método hashCode definido pela classe Object retorna inteiros distintos para objetos distintos. (Isto é tipicamente implementado convertendo o endereço interno do objeto em um inteiro, mas essa técnica de implementação não é exigida pela linguagem de programação Java.)\n\n\nPython (id)\n\n\nRetorna a "identidade" de um objeto. Isto é um inteiro (ou inteiro longo) que é garantido ser único e constante para esse objeto durante seu ciclo de vida. Dois objetos com tempo de vida não sobrepostos podem ter o mesmo valor para id().\n\nExiste alguma funcionalidade semelhante em JavaScript? De preferência padronizada, que funcionasse em todos os browsers e outros ambientes (Rhino, V8). Buscando por "identity" eu somente en

In [158]:
stemmer = SnowballStemmer('portuguese')
stopwords = nltk.corpus.stopwords.words('portuguese')

In [159]:
caracteresIgnorados = ['\t','\n']
pergunta_processed = []
for pergunta in perguntas:
    tokenizedText = nltk.word_tokenize(pergunta,language='portuguese')
    stemmedText = [stemmer.stem(t) for t in tokenizedText if t not in stopwords and not t in caracteresIgnorados]
    pergunta_processed.append(" ".join(stemmedText))
    
pergunta_processed[0]

"divers linguagens possu mei obter `` ident '' objet , i.e . númer [ inteir ] é únic cad objet , tal objet diferent ident diferent . exempl : jav ( system.identityhashcod ) tant quant razoavel prátic , métod hashcod defin class object retorn inteir distint objet distint . ( isto é tipic implement convert enderec intern objet inteir , técnic implement é exig linguag program jav . ) python ( id ) retorn `` ident '' objet . isto é inteir ( inteir long ) é garant ser únic constant objet durant cicl vid . dois objet temp vid sobrepost pod ter valor id ( ) . exist algum funcional semelh javascript ? de preferent padroniz , funcion tod browsers outr ambient ( rhin , v8 ) . busc `` identity '' soment encontr referent oper ( === ) ... sei alguns ambient suport conceit ( .net aparent faz razõ otimiz ) , geral é problem pois linguagens complet , possu conceit destrutor , finaliz e/ou referent frac . até onde saib , javascript suport , mod maneir obter valor numér unic represent objet útil ( simpl

In [160]:
cv = CountVectorizer()

X_counts = cv.fit_transform(pergunta_processed)
tf_transformer = TfidfTransformer(use_idf=True).fit(X_counts)
text_tf = tf_transformer.transform(X_counts)

print("Total de tokens: {}".format(text_tf.shape[1]))

Total de tokens: 10900


In [161]:
trainText = text_tf.toarray()

model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')
model.fit(trainText, id_repostas)
#len(trainText), len(id_repostas)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')

In [174]:
def process_new(new):
    tokenizedText = nltk.word_tokenize(new)
    stemmedText = [stemmer.stem(t) for t in tokenizedText if t not in stopwords]

    text = [word for word in stemmedText if not word in caracteresIgnorados]
    text = " ".join(text)
    cv_text = cv.transform([text])
    
    return tf_transformer.transform(cv_text)

def classify_new(new):
    
    processed_new = process_new(new).toarray()
    y = model.predict(processed_new) 
    index = int (id_repostas.index(y))
    print(index)
    #if len(y) == 1:
    #    y = y[0]
    #
    print("{}: {}".format('KNN Reposta: ',respostas[index]))

In [175]:
new = 'Diversas linguagens possuem um meio de se obter a "identidade" de um objeto, i.e. um número [inteiro] que é único para cada objeto, tal que objetos diferentes tenham identidades diferentes. Exemplos:Java (System.identityHashCode) Tanto quanto razoavelmente prático, o método hashCode definido pela classe Object retorna inteiros distintos para objetos distintos. (Isto é tipicamente implementado convertendo o endereço interno do objeto em um inteiro, mas essa técnica de implementação não é exigida pela linguagem de programação Java.)\n\n\nPython (id)\n\n\nRetorna a "identidade" de um objeto. Isto é um inteiro (ou inteiro longo) que é garantido ser único e constante para esse objeto durante seu ciclo de vida. Dois objetos com tempo de vida não sobrepostos podem ter o mesmo valor para id().\n\nExiste alguma funcionalidade semelhante em JavaScript? De preferência padronizada, que funcionasse em todos os browsers e outros ambientes (Rhino, V8). Buscando por "identity" eu somente encontro referências para o operador (===)...\nSei que alguns ambientes não suportam esse conceito (.NET aparentemente não o faz por razões de otimização), mas em geral isso não é problema pois são linguagens mais completas, que possuem os conceitos de destrutor, finalizador e/ou referências fracas. Até onde eu saiba, JavaScript não suporta isso, de modo que uma maneira de se obter um valor numérico que unicamente representa um objeto seria muito útil (uma simples referência pro objeto não serve, pois ela o impediria de ser coletado como lixo).'

In [176]:
classify_new(new);

0
KNN Reposta: : A especificação ECMA-262 não prevê essa funcionalidade na linguagem, e não tenho conhecimento de nenhum motor de JavaScript que implemente isso.
Se quiser implementar você mesmo, um ponto de partida é isto (baseado nesta resposta em inglês):
(function(){
    var id = 0;
    Object.prototype.getIdentity = function() {
        if(!this.hasOwnProperty('__identity__')) {
            this.__identity__ = ++id;
        }
        return this.__identity__;
    };
}());

Note que __identity__ é uma propriedade pública que todos os objetos que invocaram getIdentity uma vez passam a ter. Portanto nada impede que ela seja manipulada diretamente, o que potencialmente comprometeria os resultados.
Uma implementação com recursos do ECMAScript 5 (ou seja, incompatível com motores antigos como o do IE8) consegue impedir isso:
(function(){
    var id = 0;
    Object.prototype.getIdentity = function() {
        if(!this.hasOwnProperty('__identity__')) {
            Object.defineProperty(th

In [181]:
new = 'As vezes utilizo o método load e não sei quais as diferenças entre eles. Métodos: load(arg) load.apply load.call load.bind'
classify_new(new);

10
KNN Reposta: : São todo protótipos do objeto Function onde tem como objetivo executar uma função passando por ela um diferente contexto e argumentos.

call: contexto, param1, param2, param3, ...
apply: contexto, array de parametros

Já o bind ele é um pouco diferente e mais novo, nem todos browsers tem suporte, ele vai criar um "wrapper" da sua função, resultando em uma nova função, com contexto e argumentos fixos.
Por exemplo:
function foo(nome, idade) { return 'ola '+nome+', você tem '+idade}

var joao = foo.bind(window, 'joao');
joao('23'); // ola joao, você tem 23
joao('26'); // ola joao, você tem 26

// o que é o mesmo que:
var joao = function (idade) {
    return foo('joao', idade);
}
// porém evita bastante rescrita no seu código
// mantive o argumento nomeado como `idade` para fins didaticos o que
// é feito na realidade é não passar argumentos e o segundo argumento em
// foo seria `arguments[0]` dentro da função anonima do joao que é um wrapper

Voltando ao call e apply que